In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

In [5]:
train_dtypes = pd.read_csv('../input/train9_dtypes.csv')
test_dtypes = pd.read_csv('../input/test9_dtypes.csv')
train_dtypes.index = train_dtypes['TransactionID']
del train_dtypes['TransactionID']
test_dtypes.index = test_dtypes['TransactionID']
del test_dtypes['TransactionID']
train_dtypes = train_dtypes.iloc[:,0]
test_dtypes = test_dtypes.iloc[:,0]
print(type(train_dtypes),type(test_dtypes))
train_dtypes = train_dtypes.to_dict()
test_dtypes = test_dtypes.to_dict()
gc.collect()

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


21

In [6]:
%%time
train = pd.read_csv('../input/train9.csv',dtype= train_dtypes)
print("train finished")

train finished
CPU times: user 42.3 s, sys: 1.24 s, total: 43.5 s
Wall time: 43.7 s


In [7]:
%%time
test = pd.read_csv('../input/test9.csv',dtype= test_dtypes)
print("test finished")

test finished
CPU times: user 36.5 s, sys: 771 ms, total: 37.3 s
Wall time: 37.8 s


In [8]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 556) (506691, 555) (506691, 2)


In [9]:
train_Vcolumns = pd.read_csv('../input/train_Vcolumns.csv')
test_Vcolumns = pd.read_csv('../input/test_Vcolumns.csv')

In [10]:
train = pd.concat([train,train_Vcolumns],axis = 1)
test = pd.concat([test,test_Vcolumns],axis = 1)
print(train.shape,test.shape,sample_submission.shape)
gc.collect()

(590540, 696) (506691, 695) (506691, 2)


793

In [11]:
train.head()

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V302,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,uid,uid2,uid3,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card3_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,email_check,P_emaildomain_prefix,R_emaildomain_prefix,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day,device_name,card1_fq_enc,card2_fq_enc,card3_fq_enc,card5_fq_enc,C1_fq_enc,C2_fq_enc,C4_fq_enc,C5_fq_enc,C6_fq_enc,C7_fq_enc,C8_fq_enc,C9_fq_enc,C10_fq_enc,C11_fq_enc,C12_fq_enc,C13_fq_enc,C14_fq_enc,D1_fq_enc,D2_fq_enc,D3_fq_enc,D4_fq_enc,D5_fq_enc,D6_fq_enc,D8_fq_enc,addr1_fq_enc,addr2_fq_enc,dist1_fq_enc,P_emaildomain_fq_enc,R_emaildomain_fq_enc,DeviceInfo_fq_enc,device_name_fq_enc,id_30_fq_enc,id_33_fq_enc,uid_fq_enc,uid2_fq_enc,uid3_fq_enc,DT_M_total,DT_W_total,DT_D_total,uid_DT_M,uid_DT_W,uid_DT_D,C3,DeviceInfo_999999,DeviceInfo_Trident/7.0,DeviceType_mobile,Pemaildomain_mail.com,Pemaildomain_protonmail,ProductCD_DeviceInfo,Remaildomain_DeviceInfo,Remaildomain_lesscategory,Remaildomain_protonmail,TransactionAmt_decimal,V281,card1_addr1,card1_fraudulent,card2_fraudulent,card3_185.0,card3_DeviceInfo,card3_addr2,dayofyear_ischristmas,group10_kurtosis,group15_median,group4_sum,group5_kurtosis,group5_max,group5_mean,group5_median,group5_std,group6_max,group6_mean,group6_std,group7_max,hour_hourlist,id03subtractid04,id03sumid04,id05subtractid06,id09subtractid10,id09sumid10,id14_999999,id14_999999.0,id16is_null,id17_225.0,id18_15.0,id19_153.0,id19_321.0,id19_384.0,id19_427.0,id20_127.0,id20_325.0,id20_333.0,id20_401.0,id20_417.0,id20_507.0,id20_549.0,id20_597.0,id20_638.0,id31_999999,id31_firefox,id31_ie,id31_opera,id33_1280x720,id33_2208x1242,id33_secondframe,is_debit,Pemaildomain_DeviceInfo,ProductCD_card3,ProductCD_card5,ProductCD_dist2,V103,V295,V296,V309,card1_999999,dist1_108.0,dist1_86.0,group10_mean,group10_std,group10_sum,group12_mean,group12_sum,group15_max,group15_mean,group15_std,group4_kurtosis,group4_max,group6_sum,id07is_null,id19_216.0,id20is_null,id24is_null,id25is_

# Training

In [12]:
train.drop(['TransactionID'],axis=1,inplace=True)
test.drop(['TransactionID'],axis=1,inplace=True)
gc.collect()

7

In [13]:
y = train['isFraud']
del train['isFraud']

In [14]:
########################### Model params
lgb_params = {'num_leaves': 546,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.1797454081646243,
          'bagging_fraction': 0.2181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3299927210061127,
          'reg_lambda': 0.3885237330340494,
          'random_state': 42,
}

In [15]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    #params, dtrain, 1600, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500
    clf = lgb.train(lgb_params, trn_data ,2000 ,valid_sets=[trn_data, val_data], verbose_eval=200,early_stopping_rounds=500)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

Fold 1
Training until validation scores don't improve for 500 rounds.
[200]	training's auc: 0.972442	valid_1's auc: 0.903102
[400]	training's auc: 0.990987	valid_1's auc: 0.915592
[600]	training's auc: 0.997283	valid_1's auc: 0.922985
[800]	training's auc: 0.999292	valid_1's auc: 0.926956
[1000]	training's auc: 0.999828	valid_1's auc: 0.929239
[1200]	training's auc: 0.999964	valid_1's auc: 0.930407
[1400]	training's auc: 0.999994	valid_1's auc: 0.931134
[1600]	training's auc: 0.999999	valid_1's auc: 0.931372
[1800]	training's auc: 1	valid_1's auc: 0.931325
[2000]	training's auc: 1	valid_1's auc: 0.931279
Did not meet early stopping. Best iteration is:
[1999]	training's auc: 1	valid_1's auc: 0.931277
Fold 1 ROC AUC Score 0.9312774153875517

Fold 2
Training until validation scores don't improve for 500 rounds.
[200]	training's auc: 0.971933	valid_1's auc: 0.926797
[400]	training's auc: 0.991416	valid_1's auc: 0.938064
[600]	training's auc: 0.997742	valid_1's auc: 0.943157
[800]	training'

In [16]:
print(roc_auc_score(y,oof))

0.9397343426702087


In [17]:
importance['avg'] = importance.mean(axis=1)

In [18]:
importance = importance.sort_values(by='avg',ascending=False)

In [19]:
importance = importance.reset_index(drop=False)

In [31]:
importance

,index,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,avg
0,DT_D,23456,22391,23379,12050,22681,20791.4
1,uid_DT_D,21796,21830,19782,10570,22096,19214.8
2,DT_D_total,21554,21620,21018,10255,20502,18989.8
3,uid3_fq_enc,20465,20063,20136,11471,21115,18650.0
4,uid3_TransactionAmt_mean,19925,19765,19298,11183,22615,18557.2
5,uid_DT_W,19889,19889,20166,10208,20324,18095.2
6,TransactionAmt,19452,19444,19082,10341,19680,17599.8
7,uid_DT_M,18417,18063,18783,9759,16625,16329.4
8,card1_addr1,17694,17457,17319,9362,19664,16299.2
9,card1,16927,17020,17735,9726,17123,15706.2


In [25]:
importance.to_csv('../importances/importances23.csv',index=False)

In [26]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [27]:
sample_submission['isFraud'] = y_pred

In [28]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.000183
1,3663550,0.000402
2,3663551,0.000467
3,3663552,0.000440
4,3663553,0.000442


In [29]:
sample_submission.to_csv('../predictions/pred23.csv',index=False)

In [30]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred23.csv -m "pred23"

100%|███████████████████████████████████████| 14.3M/14.3M [00:33<00:00, 450kB/s]
Successfully submitted to IEEE-CIS Fraud Detection